
# <span style="color: #1f77b4;">Phase 1 Project: Aviation Safety Analysis</span>

This notebook analyzes the NTSB aviation accident dataset (1962–2023, filtered to 2010–2023) to identify low-risk aircraft for a company expanding into commercial and private aviation. 

The goal is to provide <span style="color: #2ca02c;">data-driven recommendations</span> to the head of the aviation division, supported by visualizations, and an interactive Tableau dashboard.

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display, HTML, Markdown

In [2]:
# Initialize color text 
display(HTML("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap');
    
    body {
        font-family: 'Poppins', sans-serif !important;
        line-height: 1.6;
    }
    
    .monalisa-header {
        font-family: 'Poppins', sans-serif;
        color: #1a5276;
        font-weight: 600;
        letter-spacing: 0.5px;
        margin-bottom: 15px;
        border-bottom: 2px solid #f5b041;
        padding-bottom: 5px;
    }
    
    .monalisa-highlight {
        font-family: 'Poppins', sans-serif;
        color: #27ae60;
        font-weight: 500;
        background-color: #f9f9f9;
        padding: 8px 12px;
        border-radius: 4px;
        margin: 5px 0;
    }
    
    .monalisa-warning {
        font-family: 'Poppins', sans-serif;
        color: #e67e22;
        font-weight: 500;
        background-color: #fef5e7;
        padding: 8px 12px;
        border-radius: 4px;
        margin: 5px 0;
    }
    
    .monalisa-error {
        font-family: 'Poppins', sans-serif;
        color: #e74c3c;
        font-weight: 500;
        background-color: #fdedec;
        padding: 8px 12px;
        border-radius: 4px;
        margin: 5px 0;
    }
    
    .dataframe {
        font-family: 'Poppins', sans-serif !important;
        border-radius: 8px !important;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1) !important;
    }
    
    .monalisa-quote {
        font-family: 'Poppins', sans-serif;
        font-style: italic;
        color: #5d6d7e;
        border-left: 4px solid #aed6f1;
        padding-left: 15px;
        margin: 15px 0;
    }
</style>
"""))

def display_header(text, level=2):
    display(HTML(f'<h{level} class="monalisa-header">{text}</h{level}>'))

def display_highlight(text):
    display(HTML(f'<div class="monalisa-highlight">{text}</div>'))

def display_warning(text):
    display(HTML(f'<div class="monalisa-warning">{text}</div>'))

def display_error(text):
    display(HTML(f'<div class="monalisa-error">{text}</div>'))

def display_quote(text):
    display(HTML(f'<div class="monalisa-quote">{text}</div>'))

In [3]:
# Set plot style for clear, professional visuals
plt.style.use('seaborn')
sns.set_palette('muted')

# Configure matplotlib with Poppins
plt.rcParams['font.family'] = 'Poppins'
plt.rcParams['font.weight'] = 'medium'
plt.rcParams['axes.labelweight'] = 'medium'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.titlepad'] = 15


# Set beautiful plot style
sns.set_style("whitegrid", {
    'font.family': 'Poppins',
    'grid.linestyle': ':',
    'axes.edgecolor': '0.8'
})
sns.set_palette("husl")

In [4]:
# Load the NTSB dataset 'aviation_data.csv' 
try:
    df = pd.read_csv('./data/AviationData.csv', 
                    encoding='latin-1', 
                    low_memory=False)
    display_highlight("✓ Data successfully loaded with {:,} records".format(len(df)))
except Exception as e:
    display_error(f"✗ Error loading data: {e}")

## <span style="color: #1f77b4;">Step 1: Data Cleaning</span>

We load the NTSB dataset, standardize column names, filter for recent data (2010–2023), and prepare the `aircraft` column by combining `aircraft_make` and `aircraft_model`.

In [5]:
display_quote("Refining the raw dataset into a polished imputation-ready format")
# ---------------------
# Inspect the dataset equivalent to "df.info()"
display_highlight("Initial Dataset Structure:")
info_df = pd.DataFrame({
    'Column': df.columns,
    'Non-Null Count': [df[col].notnull().sum() for col in df.columns],
    'Dtype': [df[col].dtype for col in df.columns]
})
display(info_df.style.set_properties(**{
    'font-family': 'Poppins',
    'font-size': '12px',
    'background-color': '#f9f9f9',
    'border': '1px solid #eee'
}).set_table_styles([{
    'selector': 'th',
    'props': [('background-color', '#1a5276'), ('color', 'white'), ('font-weight', '600')]
}]))


,Column,Non-Null Count,Dtype
0,Event.Id,88889,object
1,Investigation.Type,88889,object
2,Accident.Number,88889,object
3,Event.Date,88889,object
4,Location,88837,object
5,Country,88663,object
6,Latitude,34382,object
7,Longitude,34373,object
8,Airport.Code,50249,object
9,Airport.Name,52790,object


In [6]:
# Inspect for missing values
display_warning("Missing Values Summary:")
missing_data = df.isnull().sum().sort_values(ascending=False)
display(missing_data[missing_data > 0].to_frame('Missing Values').style.set_properties(**{
    'font-family': 'Poppins',
    'font-size': '12px',
    'background-color': '#f9f9f9',
    'border': '1px solid #eee'
}).set_table_styles([{
    'selector': 'th',
    'props': [('background-color', '#1a5276'), ('color', 'white'), ('font-weight', '600')]
}]))

,Missing Values
Schedule,76307
Air.carrier,72241
FAR.Description,56866
Aircraft.Category,56602
Longitude,54516
Latitude,54507
Airport.Code,38640
Airport.Name,36099
Broad.phase.of.flight,27165
Publication.Date,13771


In [7]:
# Standardize column names (lowercase, replace spaces and periods with underscores)
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '_')


In [8]:
display_highlight("Sample Data Preview:")
display(df.head().style.set_properties(**{
    'font-family': 'Poppins',
    'font-size': '12px',
    'background-color': '#f9f9f9',
    'border': '1px solid #eee'
}).set_table_styles([{
    'selector': 'th',
    'props': [('background-color', '#1a5276'), 
              ('color', 'white'),
              ('font-weight', '600')]
}]))

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,injury_severity,aircraft_damage,aircraft_category,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,schedule,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,nan,nan,nan,nan,Fatal(2),Destroyed,nan,NC6404,Stinson,108-3,No,1.000000,Reciprocating,nan,nan,Personal,nan,2.000000,0.000000,0.000000,0.000000,UNK,Cruise,Probable Cause,nan
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,nan,nan,nan,nan,Fatal(4),Destroyed,nan,N5069P,Piper,PA24-180,No,1.000000,Reciprocating,nan,nan,Personal,nan,4.000000,0.000000,0.000000,0.000000,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,nan,nan,Fatal(3),Destroyed,nan,N5142R,Cessna,172M,No,1.000000,Reciprocating,nan,nan,Personal,nan,3.000000,nan,nan,nan,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,nan,nan,nan,nan,Fatal(2),Destroyed,nan,N1168J,Rockwell,112,No,1.000000,Reciprocating,nan,nan,Personal,nan,2.000000,0.000000,0.000000,0.000000,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,nan,nan,nan,nan,Fatal(1),Destroyed,nan,N15NY,Cessna,501,No,nan,nan,nan,nan,Personal,nan,1.000000,2.000000,nan,0.000000,VMC,Approach,Probable Cause,16-04-1980


In [9]:
# Filter for 2010–2023
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df = df[df['event_date'].dt.year.between(2010, 2023)]
display_highlight(f"✓ Filtered to {len(df):,} records from 2010–2023")


In [10]:
# Create aircraft column by combining Make and Model
df['aircraft'] = df['make'].str.strip().str.title() + ' ' + df['model'].str.strip().str.upper()
df['aircraft'] = df['aircraft'].fillna('Unknown')


In [11]:
# Select relevant columns
relevant_columns = [
    'event_date', 'aircraft', 'make', 'model', 'aircraft_category',
    'injury_severity', 'total_fatal_injuries', 'total_serious_injuries',
    'total_minor_injuries', 'total_uninjured', 'broad_phase_of_flight',
    'aircraft_damage', 'event_id'
]
df = df[relevant_columns]


In [12]:
# Filter for airplanes (exclude helicopters, balloons, etc.)
df = df[df['aircraft_category'].str.contains('Airplane', na=False)]
display_highlight(f"✓ Selected {len(df):,} airplane records with {len(relevant_columns)} columns")


In [13]:
# Display cleaned dataset
display(df.head().style.set_properties(**{
    'font-family': 'Poppins',
    'font-size': '12px',
    'background-color': '#f9f9f9',
    'border': '1px solid #eee'
}).set_table_styles([{
    'selector': 'th',
    'props': [('background-color', '#1a5276'), ('color', 'white'), ('font-weight', '600')]
}]))

,event_date,aircraft,make,model,aircraft_category,injury_severity,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,broad_phase_of_flight,aircraft_damage,event_id
67590,2010-01-01 00:00:00,Cessna 150G,CESSNA,150G,Airplane,Non-Fatal,0.000000,0.000000,0.000000,2.000000,nan,Substantial,20100104X13404
67591,2010-01-01 00:00:00,Piper PA-32R-300,PIPER,PA-32R-300,Airplane,Non-Fatal,0.000000,0.000000,0.000000,1.000000,nan,Substantial,20100106X32329
67592,2010-01-02 00:00:00,Fairchild SA227,FAIRCHILD,SA227,Airplane,Non-Fatal,0.000000,0.000000,0.000000,3.000000,nan,Substantial,20100103X60452
67593,2010-01-02 00:00:00,Maule M-4-220C,MAULE,M-4-220C,Airplane,Non-Fatal,0.000000,0.000000,0.000000,3.000000,nan,Substantial,20100105X60423
67595,2010-01-02 00:00:00,Hawker Beechcraft Corp. 200,Hawker Beechcraft Corp.,200,Airplane,Non-Fatal,0.000000,0.000000,0.000000,6.000000,nan,Minor,20100714X13546


## <span style="color: #1f77b4;">Step 2: Imputation</span>

Missing values are handled using median for numerical columns (e.g., injuries) and 'Unknown' for categorical columns (e.g., `aircraft_make`, `aircraft_model`) to ensure robust analysis.

In [20]:
display_quote("Refining the cleaned dataset into a polished analysis-ready format")
# ---------------------
# Impute numerical columns with median
numeric_cols = ['total_fatal_injuries', 'total_serious_injuries', 'total_minor_injuries', 'total_uninjured']
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())
    display_highlight(f"✓ Imputed {col} with median value: {df[col].median()}")


In [15]:
# Impute categorical columns with 'Unknown'
categorical_cols = ['aircraft', 'aircraft_category', 'injury_severity', 'broad_phase_of_flight', 'aircraft_damage']
for col in categorical_cols:
    df[col] = df[col].fillna('Unknown')
    display_highlight(f"✓ Imputed {col} with 'Unknown'")


In [16]:
# Verify missing values
missing_after = df.isnull().sum()
display_warning("Missing Values After Imputation:")
display(missing_after[missing_after > 0].to_frame('Missing Values').style.set_properties(**{
    'font-family': 'Poppins',
    'font-size': '12px',
    'background-color': '#f9f9f9',
    'border': '1px solid #eee'
}).set_table_styles([{
    'selector': 'th',
    'props': [('background-color', '#1a5276'), ('color', 'white'), ('font-weight', '600')]
}]))

,Missing Values
make,1
model,8


## <span style="color: #1f77b4;">Step 3: Data Analysis</span>

We aggregate accidents and injuries by aircraft type, calculate injury rates, and analyze accident patterns by flight phase to identify low-risk aircraft.

## <span style="color: #1f77b4;">Step 4: Visualizations</span>

Three visualizations support our recommendations:
- <span style="color: #ff7f0e;">Bar Chart</span>: Top 10 aircraft by accident count.
- <span style="color: #ff7f0e;">Line Graph</span>: Accident trends over time.
- <span style="color: #ff7f0e;">Scatter Plot</span>: Accident count vs. injury rate.

## <span style="color: #1f77b4;">Step 5: Business Recommendations</span>

The following recommendations are based on our analysis to guide aircraft purchasing decisions for minimal risk.

# <span style="color: #1f77b4;">Conclusion</span>

Our analysis identifies low-risk aircraft by examining accident counts, injury rates, and temporal trends. The recommendations prioritize safety and are supported by visualizations and an interactive Tableau dashboard. For further details, contact <span style="color: #2ca02c;">[Your Name] on LinkedIn</span>.